In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_2.csv")

In [3]:
df.head()

,timestamp_local,temp,city_name,country_code,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,Gujrāt,PK,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,Gujrāt,PK,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,Gujrāt,PK,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,Gujrāt,PK,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,Gujrāt,PK,430,1782.5,60.7,2.7,533.7,377.00,253.3


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = df[['pm10']]

In [6]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [7]:
scaled = scaled.reshape((19706,))

In [8]:
scaled.shape

(19706,)

In [9]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [10]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

X_max = X_train.max()
X_min = X_train.min()

In [11]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [26]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0085 - root_mean_squared_error: 0.0900 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0706 - learning_rate: 0.0010
Epoch 2/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0050 - root_mean_squared_error: 0.0704 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0693 - learning_rate: 0.0010
Epoch 3/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0048 - root_mean_squared_error: 0.0690 - val_loss: 0.0049 - val_root_mean_squared_error: 0.0702 - learning_rate: 0.0010
Epoch 4/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0049 - root_mean_squared_error: 0.0700 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0695 - learning_rate: 0.0010
Epoch 5/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0047 - root_mean_squared_error: 0.0686 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0695 - learning_rate: 0.0010
Epoch 6/50
388/388 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0047 - root_mean_squared_er

In [13]:
forecast = model.predict(X_test)

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [14]:
poll = np.array(df["pm10"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [15]:
from sklearn.metrics import mean_squared_error

In [16]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 5.7175846


In [17]:
y_test_true.shape

(3874, 168)

In [18]:
testPredict.shape

(3874, 168)

In [19]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [20]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 1.2778
RMSE for segment 2: 1.9097
RMSE for segment 3: 2.5722
RMSE for segment 4: 3.1096
RMSE for segment 5: 3.5610
RMSE for segment 6: 3.9309
RMSE for segment 7: 4.1794
RMSE for segment 8: 4.3284
RMSE for segment 9: 4.4681
RMSE for segment 10: 4.5514
RMSE for segment 11: 4.5938
RMSE for segment 12: 4.6287
RMSE for segment 13: 4.6991
RMSE for segment 14: 4.7545
RMSE for segment 15: 4.8636
RMSE for segment 16: 4.8934
RMSE for segment 17: 4.8757
RMSE for segment 18: 4.8677
RMSE for segment 19: 4.8393
RMSE for segment 20: 4.8016
RMSE for segment 21: 4.7883
RMSE for segment 22: 4.7900
RMSE for segment 23: 4.8182
RMSE for segment 24: 4.8839
RMSE for segment 25: 4.9722
RMSE for segment 26: 5.0132
RMSE for segment 27: 5.0725
RMSE for segment 28: 5.1421
RMSE for segment 29: 5.2368
RMSE for segment 30: 5.3238
RMSE for segment 31: 5.3516
RMSE for segment 32: 5.3745
RMSE for segment 33: 5.4340
RMSE for segment 34: 5.4544
RMSE for segment 35: 5.4233
RMSE for segment 36: 5.4064
R

In [21]:
forecast

array([[0.17507191, 0.16133074, 0.14940642, ..., 0.17581843, 0.16838016,
        0.15833472],
       [0.13492468, 0.13447028, 0.13399643, ..., 0.17687625, 0.17081432,
        0.1654405 ],
       [0.07311019, 0.07140125, 0.07067638, ..., 0.07574112, 0.07541823,
        0.07525648],
       ...,
       [0.18303245, 0.18695393, 0.19574124, ..., 0.20214295, 0.18822981,
        0.17764547],
       [0.49677852, 0.4723055 , 0.3960954 , ..., 0.4689053 , 0.46135828,
        0.44151127],
       [0.06079105, 0.06347147, 0.0652651 , ..., 0.07210771, 0.07244357,
        0.07546005]], dtype=float32)

In [22]:
testPredict

array([[130.9153 , 129.7138 , 128.67117, ..., 130.98056, 130.33018,
        129.45184],
       [127.40492, 127.3652 , 127.32376, ..., 131.07306, 130.54301,
        130.07315],
       [122.00003, 121.85061, 121.78723, ..., 122.23007, 122.20184,
        122.1877 ],
       ...,
       [131.61134, 131.95422, 132.72256, ..., 133.28232, 132.06578,
        131.14032],
       [159.04445, 156.9046 , 150.241  , ..., 156.6073 , 155.9474 ,
        154.21204],
       [120.92288, 121.15725, 121.31408, ..., 121.91238, 121.94175,
        122.2055 ]], dtype=float32)

In [23]:
y_test_true

array([[132.39545 , 132.6498  , 132.90416 , ..., 126.41791 , 125.654816,
        124.89172 ],
       [126.290726, 126.290726, 126.290726, ..., 131.88672 , 130.32237 ,
        128.74533 ],
       [121.674034, 120.98725 , 120.313194, ..., 123.238365, 123.36555 ,
        123.49273 ],
       ...,
       [131.12363 , 132.14108 , 133.15852 , ..., 149.48863 , 144.47769 ,
        139.47945 ],
       [159.52325 , 156.17839 , 152.61731 , ..., 151.43451 , 149.94649 ,
        149.78116 ],
       [120.86007 , 121.41967 , 121.96655 , ..., 120.69474 , 121.71219 ,
        122.729645]], dtype=float32)

In [72]:
last_input = X_test[-1]

In [73]:
last_input.shape

(168, 1)

In [74]:
last_input = last_input.reshape(168,)

In [77]:
# predictions = []
# for _ in range(168):
#     # Predict the next value
#     print(last_input)
predicted_value = model.predict(last_input.reshape(1, 168, 1))[0]
predicted_value = predicted_value
predicted_value = predicted_value * stdop + meanop
    # predictions.append(predicted_value)
    
#     # Update the input sequence for the next iteration
#     last_input = np.append(last_input[1:], predicted_value)
# print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


In [78]:
predicted_value

array([ 88.56814 ,  87.0865  ,  85.89468 ,  85.56283 ,  85.12053 ,
        84.926125,  84.82616 ,  84.900276,  85.012024,  86.242004,
        87.66179 ,  89.064674,  93.91494 ,  98.99813 , 103.95776 ,
       104.310104, 104.34928 , 104.4447  , 102.89404 , 101.59718 ,
        99.94553 ,  96.863625,  93.75132 ,  90.66343 ,  89.166595,
        88.023994,  86.89277 ,  86.74036 ,  86.463684,  86.19579 ,
        86.0873  ,  85.94686 ,  85.92076 ,  86.63535 ,  87.637215,
        88.65329 ,  93.195015,  98.06164 , 103.030815, 104.152336,
       104.91652 , 105.34003 , 103.85257 , 102.05101 , 100.2362  ,
        96.88373 ,  93.53139 ,  90.27503 ,  88.84191 ,  87.40513 ,
        86.02289 ,  85.74353 ,  85.2294  ,  85.013855,  84.87958 ,
        85.02116 ,  85.15712 ,  86.54288 ,  88.187225,  89.803764,
        94.41786 ,  99.27209 , 104.060486, 104.77241 , 105.30119 ,
       105.5344  , 103.812996, 101.92653 ,  99.97254 ,  96.67348 ,
        93.36318 ,  90.06612 ,  88.883865,  87.41988 ,  86.257

In [35]:
y_test[-168:].ravel() * stdop + meanop

array([30.597725, 26.346886, 28.059309, 31.56474 , 28.663692, 29.550125,
       30.980501, 25.601479, 27.394485, 29.328516, 29.932901, 28.583109,
       27.172878, 28.139893, 31.504303, 29.932901, 25.601479, 26.346886,
       26.125278, 29.187494, 27.53551 , 28.663692, 30.899918, 27.253462,
       29.630709, 25.682062, 30.013485, 27.918285, 26.649078, 28.059309,
       25.299286, 29.550125, 26.870686, 28.804716, 26.2663  , 28.361502,
       26.2663  , 31.806494, 25.682062, 25.762648, 28.804716, 25.984255,
       28.663692, 26.346886, 26.649078, 28.804716, 24.553877, 26.508055,
       29.187494, 25.823086, 28.885302, 30.456701, 30.235092, 28.744278,
       24.715046, 29.187494, 25.238846, 29.630709, 27.47507 , 30.376118,
       29.711294, 24.553877, 31.423717, 29.106909, 29.026325, 25.460455,
       28.139893, 24.775486, 30.980501, 27.01171 , 27.696678, 24.997093,
       25.460455, 27.777262, 29.106909, 27.8377  , 28.965885, 28.059309,
       29.550125, 29.932901, 28.583109, 30.013485, 

In [24]:
model.save("pm10_1dcnn.keras")